## Agent Basics

In [1]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x+y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x*y

@tool
def exponentiate(x: float, y: float) -> float:
    """Exponentiate 'x' to the power of 'y'."""
    return x**y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y-x

With the @tool decorator our function is turned into a StructuredTool object

In [2]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7f7eae2ce020>)

In [3]:
print(f"{add.name}\n{add.description}")

add
Add 'x' and 'y'.


In [4]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to below

In [5]:
import json 

llm_output_string = "{\"x\":5, \"y\":2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.

In [6]:
exponentiate.func(**llm_output_dict)

25

## Creating an Agent

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

In [16]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwen3:0.6b", temperature=0.0)

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/tmp/ipykernel_124913/4074810399.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we can initialize our agent that needs 

1. llm
2. tools
3. prompt
4. memory

In [17]:
from langchain.agents import create_tool_calling_agent
tools = [add, subtract, exponentiate, multiply]
agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

Our agent by itself is like on-step of our agent execution loop. So, if we call agent.invoke method it will get the LLM to geenrate a single response and go no further, so no tools will be executed, and no next iterations will be performed

In [ ]:
agent.invoke({
    "input": "What is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": [] # agent appends its steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-05-18T21:39:56.879918955Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3130729732, 'load_duration': 2319377267, 'prompt_eval_count': 355, 'prompt_eval_duration': 364436420, 'eval_count': 108, 'eval_duration': 435173587, 'model_name': 'qwen3:0.6b'}, id='run--eb729e63-35cf-425e-88c5-59d1f05d1e36-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'b02a1db7-b17b-4e4d-80d3-fd896f2dd701', 'type': 'tool_call'}], usage_metadata={'input_tokens': 355, 'output_tokens': 108, 'total_tokens': 463})], tool_call_id='b02a1db7-b17b-4e4d-80d3-fd896f2dd701')]

We know LLM wants to use a tool and has parameters for it. But we do not have the answer, the reason is LLM cannot execute the function -> we need agent executor for it

## Agent Executor

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memeory=memory,
    verbose=True
)

In [20]:
agent_executor.invoke({
    "input": "What is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.17599999999999<think>
Okay, the user asked, "What is 10.7 multiplied by 7.68?" I need to use the multiply function. Let me check the tools provided. Yes, there's a multiply function that takes two numbers. So I'll call multiply with x=10.7 and y=7.68. Then, the response should be the product, which is approximately 82.18. I should present that clearly.
</think>

The result of multiplying 10.7 by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'What is 10.7 multiplied by 7.68?',
 'chat_history': [],
 'output': '<think>\nOkay, the user asked, "What is 10.7 multiplied by 7.68?" I need to use the multiply function. Let me check the tools provided. Yes, there\'s a multiply function that takes two numbers. So I\'ll call multiply with x=10.7 and y=7.68. Then, the response should be the product, which is approximately 82.18. I should present that clearly.\n</think>\n\nThe result of multiplying 10.7 by 7.68 is approximately 82.18.'}

In [21]:
agent_executor.invoke({
    "input": "What is 10.7 multiplied by 7.68, subtracted by 20.18, and added by 30?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.17599999999999
Invoking: `subtract` with `{'x': 82.176, 'y': 20.18}`


-61.996
Invoking: `add` with `{'x': 62, 'y': 30}`


92.0<think>
Okay, let's see. The user asked for the result of 10.7 multiplied by 7.68, subtracted by 20.18, and added by 30. I need to calculate each step and then combine them.

First, multiply 10.7 and 7.68. Let me do that. 10.7 times 7.68. Let me compute that. 10 times 7.68 is 76.8, and 0.7 times 7.68 is 5.376. Adding those gives 82.176. So the first part is 82.176.

Next, subtract 20.18 from that result. So 82.176 minus 20.18. Let me subtract. 82.176 - 20 is 62.176, then subtract 0.18 more, which gives 62.0. So that's -61.996.

Then, add 30 to the result. 62.0 plus 30 is 92.0. So the final result is 92.0.

Wait, but the user might want the exact decimal places. Let me check the original numbers. The original numbers are given as 10.7, 7.68, 20.18, and 30. All are pr

{'input': 'What is 10.7 multiplied by 7.68, subtracted by 20.18, and added by 30?',
 'chat_history': [],
 'output': "<think>\nOkay, let's see. The user asked for the result of 10.7 multiplied by 7.68, subtracted by 20.18, and added by 30. I need to calculate each step and then combine them.\n\nFirst, multiply 10.7 and 7.68. Let me do that. 10.7 times 7.68. Let me compute that. 10 times 7.68 is 76.8, and 0.7 times 7.68 is 5.376. Adding those gives 82.176. So the first part is 82.176.\n\nNext, subtract 20.18 from that result. So 82.176 minus 20.18. Let me subtract. 82.176 - 20 is 62.176, then subtract 0.18 more, which gives 62.0. So that's -61.996.\n\nThen, add 30 to the result. 62.0 plus 30 is 92.0. So the final result is 92.0.\n\nWait, but the user might want the exact decimal places. Let me check the original numbers. The original numbers are given as 10.7, 7.68, 20.18, and 30. All are precise to two decimal places except maybe the 10.7 which is one decimal place. But when multiplying